
# Aggregated & Preprocessed Data


**Author:** Sahil Makhamale


**Contents:**

- Load raw dataset
- Data cleaning: missing values, duplicates, dtype corrections
- Hourly aggregation per junction
- Preprocessing: scaling, time features, lag features, weekend/event indicators
- Save aggregated CSV for submission


> This notebook prepares `IntegratedDataset_SahilMakhamale_hourly.csv` suitable for model training and submission.


In [29]:
# Load raw dataset
import pandas as pd

# --- IMPORTANT: Please replace 'your_file_path_here.csv' with the actual path to your uploaded file ---
# After uploading the file using the 'Files' tab on the left, right-click the file and select 'Copy path'.
# Example of using a raw string for a Windows path:
file_path = r'/content/IntegratedDataset_SahilMakhamale.csv.csv' # <<<--- REPLACE WITH YOUR UPLOADED FILE'S PATH!

try:
    df = pd.read_csv(file_path)
    print("Loaded rows, cols:", df.shape)
    print(df.head())
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please ensure the file is uploaded and the path is correct.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Loaded rows, cols: (48120, 4)
        DateTime  Junction  Vehicles           ID
0  01/11/15 0:00         1        15  20151101001
1  01/11/15 1:00         1        13  20151101011
2  01/11/15 2:00         1        10  20151101021
3  01/11/15 3:00         1         7  20151101031
4  01/11/15 4:00         1         9  20151101041


In [30]:

# Basic cleaning
# 1) Drop exact duplicate rows
df = df.drop_duplicates()

# 2) Heuristics to find datetime, junction, and target columns
datetime_cols = [c for c in df.columns if 'date' in c.lower() or 'time' in c.lower() or 'timestamp' in c.lower() or 'datetime' in c.lower()]
if not datetime_cols:
    for c in df.columns:
        parsed = pd.to_datetime(df[c], errors='coerce')
        if parsed.notna().sum() / len(parsed) > 0.6:
            datetime_cols.append(c)
            break
datetime_col = datetime_cols[0] if datetime_cols else None

loc_cols = [c for c in df.columns if any(k in c.lower() for k in ['junction','location','site','station','sensor','id'])]
junction_col = loc_cols[0] if loc_cols else None

num_cols = df.select_dtypes(include=['number']).columns.tolist()
target_cols = [c for c in df.columns if any(k in c.lower() for k in ['volume','traffic','count','flow','vehicles'])]
target_col = target_cols[0] if target_cols else (num_cols[0] if num_cols else None)

print("Detected -> datetime:", datetime_col, "junction:", junction_col, "target:", target_col)

# Convert datetime
if datetime_col:
    df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce')
else:
    raise ValueError("No datetime-like column detected. Please set datetime column and re-run.")

# Drop rows without datetime or target
if target_col:
    df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
    df = df.dropna(subset=[datetime_col, target_col])
else:
    df = df.dropna(subset=[datetime_col])

# Basic imputation (forward fill) for non-target columns
df = df.fillna(method='ffill')
print("After cleaning rows, cols:", df.shape)
df.head()


Detected -> datetime: DateTime junction: Junction target: Vehicles


/tmp/ipython-input-72863933.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[datetime_col] = pd.to_datetime(df[datetime_col], errors='coerce')


After cleaning rows, cols: (48120, 4)


/tmp/ipython-input-72863933.py:38: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


,DateTime,Junction,Vehicles,ID
0,2015-01-11 00:00:00,1,15,20151101001
1,2015-01-11 01:00:00,1,13,20151101011
2,2015-01-11 02:00:00,1,10,20151101021
3,2015-01-11 03:00:00,1,7,20151101031
4,2015-01-11 04:00:00,1,9,20151101041


In [34]:

# ensure the DateTime column is parsed as datetime
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')

# drop rows where DateTime could not be parsed
df = df.dropna(subset=['DateTime'])

# if you have a junction column, ensure it's string
df['junction'] = df['junction'].astype(str)   # or set to 'ALL' if no such column

# set the datetime column as the index
df = df.set_index('DateTime')

# now groupby per junction and hourly resample using pd.Grouper(freq='H')
df_hourly = df.groupby(['junction', pd.Grouper(freq='H')])['Vehicles'].sum().reset_index().rename(columns={'DateTime':'datetime','Vehicles':'traffic_volume'})

# sort and inspect
df_hourly = df_hourly.sort_values(['junction','datetime']).reset_index(drop=True)
print(df_hourly.shape)
df_hourly.head()



(48120, 3)


/tmp/ipython-input-1131044270.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.groupby(['junction', pd.Grouper(freq='H')])['Vehicles'].sum().reset_index().rename(columns={'DateTime':'datetime','Vehicles':'traffic_volume'})


,junction,datetime,traffic_volume
0,1,2015-01-11 00:00:00,15
1,1,2015-01-11 01:00:00,13
2,1,2015-01-11 02:00:00,10
3,1,2015-01-11 03:00:00,7
4,1,2015-01-11 04:00:00,9


In [32]:

# Feature engineering: time features, lag features, weekend indicator
df_hourly['datetime'] = pd.to_datetime(df_hourly['datetime'])
# Fix for 'H' deprecation warning and ValueError: cannot insert junction, already exists
# Use 'h' for frequency and reset all index levels after apply.
df_hourly = df_hourly.set_index('datetime').groupby('junction').apply(lambda g: g['traffic_volume'].asfreq('h', fill_value=0)).reset_index()
df_hourly['hour'] = df_hourly['datetime'].dt.hour
df_hourly['day_of_week'] = df_hourly['datetime'].dt.dayofweek
df_hourly['month'] = df_hourly['datetime'].dt.month
df_hourly['is_weekend'] = df_hourly['day_of_week'].isin([5,6]).astype(int)

# Lag features (1-24 hours)
for lag in range(1,25):
    df_hourly[f'lag_{lag}'] = df_hourly.groupby('junction')['traffic_volume'].shift(lag)

# Drop rows with NaN in lag features
df_hourly = df_hourly.dropna().reset_index(drop=True)
print("After feature engineering shape:", df_hourly.shape)
df_hourly.head()


/tmp/ipython-input-1936189371.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_hourly = df_hourly.set_index('datetime').groupby('junction').apply(lambda g: g['traffic_volume'].asfreq('h', fill_value=0)).reset_index()


After feature engineering shape: (84360, 31)


,junction,datetime,traffic_volume,hour,day_of_week,month,is_weekend,lag_1,lag_2,lag_3,...,lag_15,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24
0,1,2015-01-13 00:00:00,0,0,1,1,0,26.0,24.0,30.0,...,21.0,16.0,12.0,11.0,11.0,14.0,13.0,18.0,20.0,21.0
1,1,2015-01-13 01:00:00,0,1,1,1,0,0.0,26.0,24.0,...,30.0,21.0,16.0,12.0,11.0,11.0,14.0,13.0,18.0,20.0
2,1,2015-01-13 02:00:00,0,2,1,1,0,0.0,0.0,26.0,...,29.0,30.0,21.0,16.0,12.0,11.0,11.0,14.0,13.0,18.0
3,1,2015-01-13 03:00:00,0,3,1,1,0,0.0,0.0,0.0,...,28.0,29.0,30.0,21.0,16.0,12.0,11.0,11.0,14.0,13.0
4,1,2015-01-13 04:00:00,0,4,1,1,0,0.0,0.0,0.0,...,31.0,28.0,29.0,30.0,21.0,16.0,12.0,11.0,11.0,14.0


In [33]:

# Scaling traffic_volume (optional) - StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_hourly['traffic_scaled'] = scaler.fit_transform(df_hourly[['traffic_volume']])

# Save final processed dataset (this is the same file but with features)
final_csv = r"/content/IntegratedDataset_SahilMakhamale_hourly_with_features.csv"
df_hourly.to_csv(final_csv, index=False)
print("Saved final processed CSV with features to:", final_csv)
df_hourly.head()


Saved final processed CSV with features to: /content/IntegratedDataset_SahilMakhamale_hourly_with_features.csv


,junction,datetime,traffic_volume,hour,day_of_week,month,is_weekend,lag_1,lag_2,lag_3,...,lag_16,lag_17,lag_18,lag_19,lag_20,lag_21,lag_22,lag_23,lag_24,traffic_scaled
0,1,2015-01-13 00:00:00,0,0,1,1,0,26.0,24.0,30.0,...,16.0,12.0,11.0,11.0,14.0,13.0,18.0,20.0,21.0,-0.671922
1,1,2015-01-13 01:00:00,0,1,1,1,0,0.0,26.0,24.0,...,21.0,16.0,12.0,11.0,11.0,14.0,13.0,18.0,20.0,-0.671922
2,1,2015-01-13 02:00:00,0,2,1,1,0,0.0,0.0,26.0,...,30.0,21.0,16.0,12.0,11.0,11.0,14.0,13.0,18.0,-0.671922
3,1,2015-01-13 03:00:00,0,3,1,1,0,0.0,0.0,0.0,...,29.0,30.0,21.0,16.0,12.0,11.0,11.0,14.0,13.0,-0.671922
4,1,2015-01-13 04:00:00,0,4,1,1,0,0.0,0.0,0.0,...,28.0,29.0,30.0,21.0,16.0,12.0,11.0,11.0,14.0,-0.671922
